In [1]:
import os
import numpy as np
import plotly.graph_objs as go
import plotly
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid, save_image
from torch.nn import functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from scipy.stats import norm
import scipy.io
import os
from math import ceil
import joblib
from sklearn import preprocessing
import plotly.express as px

device = ( 'cuda' if torch.cuda.is_available() else 'cpu' )

In [3]:
# Autoencoder

# Hyperparameters
batch_size = 128
device = device = ( 'cuda' if torch.cuda.is_available() else 'cpu' )

img_size = 138    # 28x28x1
hidden_dim = 64  # hidden layer dim
#z_dim = 20        # latent space dimension (encoder)
z_dim = 2        # latent space dimension (encoder)

epochs = 10

# model architecture
class VAE(nn.Module):
  def __init__(self):
    super(VAE, self).__init__()

    # encoder
    self.fc1 = nn.Linear(img_size, hidden_dim)
    self.fc2_mean = nn.Linear(hidden_dim, z_dim)
    self.fc2_logvar = nn.Linear(hidden_dim, z_dim)
    #decoder
    self.fc3 = nn.Linear(z_dim, hidden_dim)
    self.fc4 = nn.Linear(hidden_dim, img_size)

  def encode(self, x):
    h = F.relu(self.fc1(x))
    mu = self.fc2_mean(h)         # compute mean of latent
    logvar = self.fc2_logvar(h)   # compute logvar of latent
    return mu, logvar

  def reparameterize(self, mu, logvar):
    # p(z|x) = mu + std * eps
    std = torch.exp(logvar/2)
    eps = torch.randn_like(std)
    return (mu + eps * std)

  def decode(self, z):
    h = F.relu(self.fc3(z))
    out = torch.sigmoid(self.fc4(h))
    return out

  def forward(self, x):
    # batch_size x 1 x 28 x 28 -> batch_size x 784
    #print(x.view(-1, img_size).shape)
    mu, logvar = self.encode(x.view(-1, img_size))
    z = self.reparameterize(mu, logvar)
    reconstructed = self.decode(z)
    return reconstructed, mu, logvar

# Initialize model, optimizer
audioVae = VAE().to(device)
audioVae.load_state_dict(torch.load('/content/drive/MyDrive/video_feat/audio_only/autoenc_audio_only.pth'))

<All keys matched successfully>

In [4]:
encoder = preprocessing.LabelEncoder()
encoder.classes_ = np.load('/content/drive/MyDrive/video_feat/audio_only/classes.npy')

# these are the encoded video clips
songs_z = pd.read_csv('/content/drive/MyDrive/video_feat/audio_only/audio_z.csv')
songs_z.head()

labels2 = encoder.inverse_transform(songs_z.labels)
#print(labels)
songs_z_normal = songs_z
songs_z_normal.labels = labels2
songs_z_normal.head()
#songs_z = audioVae.encode(songs)

#test_song = torch.randn(138)

#test_song_z = audioVae.encode(test_song)

,x,y,labels
0,25.158928,1.049391,Calum-Scott-Dancing-On-My-Own
1,-2.364750,0.553579,Billie-Eilish-everything-i-wanted
2,-5.552360,0.839206,24kGoldn-Mood
3,0.643493,0.635634,Alec-Benjamin-Let-Me-Down-Slowly
4,-1.848031,0.663291,Axwell-and-Ingrosso-More-Than-You-Know


In [5]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=5)

neigh.fit(songs_z[['x', 'y']].values)


#nearest = neigh.kneighbors([[0,1]], return_distance=False).reshape(-1)
#y3 = le.inverse_transform(y2)

#most_similar = []
#for k in nearest:
#  most_similar.append(y3[k])

#print(most_similar)

nearest = neigh.kneighbors([[0,1]], return_distance=False).reshape(-1)
#y3 = le.inverse_transform(y2)

most_similar = []
for k in nearest:
  most_similar.append(labels2[k])

print(most_similar)

['Dua-Lipa-Levitating', 'Imagine-Dragons-Believer', 'Kendrick-Lamar-HUMBLE', '5-Seconds-of-Summer-Youngblood', 'SZA-Good-Days']


In [6]:
# correct solution:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference


neighbor_distance, neighbor_index = neigh.kneighbors([[0,1]], return_distance=True)
ha = (neighbor_distance.reshape(-1, 1)+100)

neighbor_distance = softmax(ha).round(3)*100
#print(neighbor_distance)
neighbor_index = encoder.inverse_transform(neighbor_index.reshape(-1))
#print(neighbor_index)

for i in range(len(neighbor_index)):
  print(f'track{i+1}: {neighbor_index[i]}, probability: {neighbor_distance[i]}')


track1: Farruko-Pepas, probability: [17.2]
track2: Imagine-Dragons-x-J-I-D-Enemy, probability: [17.2]
track3: Jaymes-Young-Infinity, probability: [19.8]
track4: Billie-Eilish-everything-i-wanted, probability: [20.7]
track5: Rosa-Linn-Snap, probability: [25.1]


In [ ]:
!git clone https://github.com/tyiannak/pyAudioAnalysis.git

!pip install pyAudioAnalysis

#!pip install numpy
#!pip install matplotlib
#!pip install scipy
!pip install sklearn
#!pip install hmmlearn
!pip install simplejson
!pip install eyed3
!pip install pydub

In [8]:
from pyAudioAnalysis import MidTermFeatures as aF
#from pyAudioAnalysis import audioTrainTest as aFa
import os
import numpy as np
import plotly.graph_objs as go
import plotly

In [9]:
fpath = '/content/drive/MyDrive/multi_modal/test'

classes = os.listdir(fpath)
print(classes)
#print(dirs)
dirs= []
for file in os.listdir(fpath):
    d = os.path.join(fpath, file)
    if os.path.isdir(d):
        dirs.append(d)

print(len(dirs))

class_names = [os.path.basename(d) for d in classes]

m_win, m_step, s_win, s_step = 1, 1, 0.1, 0.05

features = []
class_n = []
i= 0
for d in dirs: # get feature matrix for each directory (class)
    name = os.path.basename(d)
    class_n.append(name)

    f, files, fn = aF.directory_feature_extraction(d, m_win, m_step,
                                                   s_win, s_step)
    features.append(f)
    print(i)
    i+=1

['.ipynb_checkpoints', 'test1', 'g.py']
2
0
Analyzing file 1 of 1: /content/drive/MyDrive/multi_modal/test/test1/test1.wav


/usr/local/lib/python3.10/dist-packages/pyAudioAnalysis/audioBasicIO.py:99: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sampling_rate, signal = wavfile.read(input_file) # from scipy.io


Feature extraction complexity ratio: 19.6 x realtime
1


In [10]:
arr = np.array(features[1])
print(arr.shape)
print(arr[1])
testdat = torch.tensor(arr,dtype=torch.float32)

_, mu, logvar = audioVae(testdat)
z = audioVae.reparameterize(mu, logvar).data.cpu().numpy()
print(z)

(138,)
0.15477497346100064
[[0.23589385 0.39585823]]


In [11]:
# correct solution:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference


neighbor_distance, neighbor_index = neigh.kneighbors(z, return_distance=True)
n_dist = neighbor_distance # for circle color
ha = (neighbor_distance.reshape(-1, 1)+100)

neighbor_distance = softmax(ha).round(3)*100
#print(neighbor_distance)
neighbor_index = encoder.inverse_transform(neighbor_index.reshape(-1))
#print(neighbor_index)

for i in range(len(neighbor_index)):
  print(f'track{i+1}: {neighbor_index[i]}, probability: {neighbor_distance[i]}')

track1: Harry-Styles-Golden, probability: [17.5]
track2: Taylor-Swift-You-Need-To-Calm-Down, probability: [18.3]
track3: Anne-Marie-2002, probability: [18.8]
track4: Farruko-Pepas, probability: [20.4]
track5: DJ-Khaled-I-m-The-One, probability: [25.]


In [12]:
def extract_features(fpath):
    classes = os.listdir(fpath)
    #print(classes)
    #print(dirs)
    dirs= []
    for file in os.listdir(fpath):
        d = os.path.join(fpath, file)
        if os.path.isdir(d):
            dirs.append(d)

    #print(len(dirs))

    class_names = [os.path.basename(d) for d in classes]

    m_win, m_step, s_win, s_step = 1, 1, 0.1, 0.05

    features = []
    class_n = []
    i= 0
    for d in dirs: # get feature matrix for each directory (class)
        name = os.path.basename(d)
        class_n.append(name)

        f, files, fn = aF.directory_feature_extraction(d, m_win, m_step,
                                                      s_win, s_step)
        features.append(f)
        #print(i)
        i+=1

    arr = np.array(features[1])
    testdat = torch.tensor(arr,dtype=torch.float32)

    return testdat



In [13]:
def similarityAudio(fpath):

  audio_extr_f = extract_features(fpath)

  _, mu, logvar = audioVae(audio_extr_f)

  epsilon = 0.1
  std = torch.exp(logvar/2)
  eps = torch.randn_like(std)
  z = (mu + epsilon * std).data.cpu().numpy()
  #z = audioVae.reparameterize(mu, logvar).data.cpu().numpy()
  neighbor_distance, neighbor_index = neigh.kneighbors(z, return_distance=True)
  neighbor_color = neighbor_index
  #print(neighbor_color)
  plot_radius = float(np.array(np.max(neighbor_distance)))
  #print(f'pr {plot_radius}')
  ha = (neighbor_distance.reshape(-1, 1)+100)

  neighbor_distance = softmax(ha).round(3)*100
  neighbor_distance = softmax(100 - neighbor_distance).round(3)*100
  #print(neighbor_distance)
  neighbor_index = encoder.inverse_transform(neighbor_index.reshape(-1))
  #print(neighbor_index)
  print()
  print()
  for i in range(len(neighbor_index)):
    print(f'track{i+1}: {neighbor_index[i]}, probability: {neighbor_distance[i]}')


  #fig = px.scatter(x=songs_z.iloc[:, 0], y=songs_z.iloc[:, 1])
  #fig = px.scatter(x=z[:, 0], y=z[:, 1])
  #fig.show()

  import plotly.express as px
  import plotly.graph_objects as go

  fig1 = px.scatter(x=songs_z.iloc[:, 0], y=songs_z.iloc[:, 1])
  fig2 = px.scatter(x=z[:, 0], y=z[:, 1], color_discrete_sequence=['red'])
  #print(list(neighbor_color.reshape(-1)))
  z4 = songs_z.iloc[list(neighbor_color.reshape(-1))]
  #print(z4)
  fig4 = px.scatter(x=z4['x'], y=z4['y'], color =neighbor_index) #color_discrete_sequence=['LightGreen'])
  fig4.update_traces(
    marker=dict(size=8, symbol="star-diamond", line=dict(width=2, color="DarkSlateGrey")),
    selector=dict(mode="markers"),
  )


  fig3 = go.Figure(data=fig1.data + fig2.data + fig4.data)
  print(f'lp {z[:, 0]- plot_radius}')
  fig3.add_shape(type="circle", xref="x", yref="y", x0=float(z[:, 0])-plot_radius,
                 y0=float(z[:, 1])-plot_radius, x1=float(z[:, 0])+plot_radius, y1=float(z[:, 1])+plot_radius,
                 line_color="LightGreen", fillcolor="LightSalmon", opacity=0.2,)

  # Set figure size
  #fig3.update_layout(width=400, height=400)
  fig3.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
  )

  fig3.show()

In [14]:
print(list(neighbor_index.reshape(-1)))

['Harry-Styles-Golden', 'Taylor-Swift-You-Need-To-Calm-Down', 'Anne-Marie-2002', 'Farruko-Pepas', 'DJ-Khaled-I-m-The-One']


Audio Based Similarity Retrieval

In [15]:
similarityAudio(fpath)

Analyzing file 1 of 1: /content/drive/MyDrive/multi_modal/test/test1/test1.wav


/usr/local/lib/python3.10/dist-packages/pyAudioAnalysis/audioBasicIO.py:99: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sampling_rate, signal = wavfile.read(input_file) # from scipy.io


Feature extraction complexity ratio: 19.9 x realtime


track1: Ti-sto-The-Business, probability: [53.2]
track2: Kygo-Selena-Gomez-It-Ain-t-Me, probability: [32.3]
track3: Glass-Animals-Heat-Waves, probability: [14.5]
track4: Taylor-Swift-Lover, probability: [0.]
track5: Taylor-Swift-cardigan, probability: [0.]
lp [0.6544049]
